# BrainGraph 86-node set

This notebook:
- Loads `.graphml` brain connectomes (86-node set) with **NetworkX**.
- Inspects node/edge attributes.
- Creates two basic models on a subset of the data to make sure things run

> Update `DATA_DIR` to your folder (defaults to `data/HCP/86_nodes`).
> Update `PHENO_PATH` to the path to the subjects csv (defaults to `data/HCP/HCP_phenotypes.csv`).

In [1]:
from pathlib import Path
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch_geometric.data import Data


Matplotlib is building the font cache; this may take a moment.


In [2]:
DATA_DIR = Path("data/HCP/86_nodes")

assert DATA_DIR.exists(), f"DATA_DIR does not exist: {DATA_DIR}"
files = sorted(DATA_DIR.glob("*.graphml"))
print(f"Found {len(files)} .graphml files in {DATA_DIR}")
print("First 5 files:")
for f in files[:5]:
    print("  ", f.name)

Found 1064 .graphml files in data/HCP/86_nodes
First 5 files:
   100206_repeated10_scale33.graphml
   100307_repeated10_scale33.graphml
   100408_repeated10_scale33.graphml
   100610_repeated10_scale33.graphml
   101006_repeated10_scale33.graphml


In [4]:
PHENO_PATH = Path("data/HCP/HCP_phenotypes.csv")

assert PHENO_PATH.exists(), f"PHENO_PATH does not exist: {PHENO_PATH}"
pheno = pd.read_csv(PHENO_PATH)

print("Age_in_Yrs" in pheno.columns)

True


In [5]:
pheno.head()

,Subject,Release,Acquisition,Gender,Age,Age_in_Yrs,HasGT,ZygositySR,ZygosityGT,Family_ID,...,MOV2_TRACKFRAC,MOV2_TRFRAC,MOV3_TRACKFRAC,MOV3_TRFRAC,MOV4_TRACKFRAC,MOV4_TRFRAC,MOV_EYETRACK_COMPL,REST_TRACKFRAC_MIN,REST_TRFRAC_MIN,REST_EYETRACK_COMPL
0,100004,S900,Q06,M,22-25,24,True,NotTwin,,52259_82122,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100206,S900,Q11,M,26-30,27,True,NotTwin,,56037_85858,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100307,Q1,Q01,F,26-30,27,True,NotMZ,MZ,51488_81352,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100408,Q3,Q03,M,31-35,33,True,MZ,MZ,51730_81594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100610,S900,Q08,M,26-30,27,True,NotMZ,DZ,52813_82634,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Load a single example graph & peek
g = nx.read_graphml(files[0])
print("loaded:", files[0].name)
print("nodes:", g.number_of_nodes(), "edges:", g.number_of_edges())

# peek a node
n0 = list(g.nodes())[0]
print("\nexample node:", n0)
print("node attrs:", g.nodes[n0])

# peek an edge
e0 = list(g.edges())[0]
print("\nexample edge:", e0)
print("edge attrs:", g.edges[e0])

loaded: 100206_repeated10_scale33.graphml
nodes: 83 edges: 725

example node: 1
node attrs: {'dn_position_x': 34.0889628925, 'dn_position_y': 83.5585156993, 'dn_position_z': 8.72454804948, 'dn_correspondence_id': '1', 'dn_region': 'cortical', 'dn_fsname': 'lateralorbitofrontal', 'dn_name': 'rh.lateralorbitofrontal', 'dn_hemisphere': 'right'}

example edge: ('1', '34')
edge attrs: {'fiber_length_mean': 13.3253736496, 'FA_mean': 0.202591825277, 'number_of_fibers': 10.125}


#### Following cell creates a smaller version of subjects csv file with just age and id, skip and run next cell if this has already been run

In [7]:
# keep only subject ID and age
meta = pheno[['Subject', 'Age_in_Yrs']].copy()

# rename columns
meta = meta.rename(columns={
    'Subject': 'subject_id',
    'Age_in_Yrs': 'age'
})

meta['subject_id'] = meta['subject_id'].astype(str)  # raw data uses integers for id

meta.to_csv("data/HCP/HCP_subjects_age_only.csv", index=False)
meta.shape


(1206, 2)

In [8]:
pheno_meta = pd.read_csv(
    "data/HCP/HCP_subjects_age_only.csv",
    dtype={"subject_id": str}
)

In [9]:
print(type(pheno_meta['subject_id'][0]))

<class 'str'>


In [ ]:
# Construct training dataset

def graph_to_data(graph_path):
    G = nx.read_graphml(graph_path)
    nodes = list(G.nodes())
    idx = {n: i for i, n in enumerate(nodes)}

    coords = np.zeros((len(nodes), 3), dtype=np.float32)
    for n in nodes:
        attrs = G.nodes[n]
        coords[idx[n]] = [float(attrs['dn_position_x']), float(attrs['dn_position_y']), float(attrs['dn_position_z'])]
    x = torch.tensor(coords, dtype=torch.float32)

    src, dst, ew = [], [], []
    for u, v, d in G.edges(data=True):
        ui, vi = idx[u], idx[v]
    
        # use number_of_fibers as weights
        w = float(d['number_of_fibers'])
    
        # both directions for undirected graph
        src += [ui, vi]
        dst += [vi, ui]
        ew  += [w, w]

    edge_index  = torch.tensor([src, dst], dtype=torch.long)
    edge_weight = torch.tensor(ew, dtype=torch.float32)

    # Label (age)
    filename = graph_path.name
    subject_id = filename.split('_')[0]
    age = pheno_meta.loc[pheno_meta.subject_id == subject_id, "age"].values[0]
    y = torch.tensor([age], dtype=torch.float32)

    return Data(x=x, edge_index=edge_index, edge_weight=edge_weight, y=y)

In [14]:
# Use a small subset first to test
N = 200
subset_files = files[:N]

data_list = [graph_to_data(p) for p in subset_files]
len(data_list)


200

In [15]:
from sklearn.model_selection import train_test_split

train_idx, test_idx = train_test_split(range(len(data_list)), test_size=0.2, random_state=42)
train_idx, val_idx = train_test_split(train_idx, test_size=0.2, random_state=42)

train_data = [data_list[i] for i in train_idx]
val_data   = [data_list[i] for i in val_idx]
test_data  = [data_list[i] for i in test_idx]


In [16]:
import torch.nn as nn
from torch_geometric.nn import SAGEConv, GCNConv, global_mean_pool

In [17]:
# Basic models

class GraphSAGERegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SAGEConv(3, 32)
        self.conv2 = SAGEConv(32, 32)
        self.readout = nn.Linear(32, 1)

    def forward(self, data):
        x = torch.relu(self.conv1(data.x, data.edge_index))
        x = torch.relu(self.conv2(x, data.edge_index))
        # single-graph case: batch is all zeros
        batch = torch.zeros(x.size(0), dtype=torch.long, device=x.device)
        hg = global_mean_pool(x, batch)
        return self.readout(hg).squeeze(-1)

class GCNRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(3, 32)
        self.conv2 = GCNConv(32, 32)
        self.readout = nn.Linear(32, 1)

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight
        x = torch.relu(self.conv1(x, edge_index, edge_weight))
        x = torch.relu(self.conv2(x, edge_index, edge_weight))
        batch = torch.zeros(x.size(0), dtype=torch.long, device=x.device)
        hg = global_mean_pool(x, batch)
        return self.readout(hg).squeeze(-1)


In [18]:
# Train basic model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GraphSAGERegressor().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.L1Loss()

def run_batch(data_list, train=True):
    total_loss = 0
    for data in data_list:
        data = data.to(device)
        pred = model(data)
        loss = loss_fn(pred, data.y)
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_list)

for epoch in range(1, 100):
    train_loss = run_batch(train_data, train=True)
    val_loss = run_batch(val_data, train=False)
    if epoch % 5 == 0:
        print(f"epoch {epoch:3d} | train MAE {train_loss:.3f} | val MAE {val_loss:.3f}")


epoch   5 | train MAE 3.764 | val MAE 2.737
epoch  10 | train MAE 3.629 | val MAE 3.092
epoch  15 | train MAE 3.699 | val MAE 2.848
epoch  20 | train MAE 3.645 | val MAE 2.804
epoch  25 | train MAE 3.631 | val MAE 2.748
epoch  30 | train MAE 3.580 | val MAE 2.806
epoch  35 | train MAE 3.621 | val MAE 2.722
epoch  40 | train MAE 3.638 | val MAE 2.815
epoch  45 | train MAE 3.603 | val MAE 2.721
epoch  50 | train MAE 3.613 | val MAE 2.775
epoch  55 | train MAE 3.585 | val MAE 2.731
epoch  60 | train MAE 3.582 | val MAE 2.735
epoch  65 | train MAE 3.606 | val MAE 2.721
epoch  70 | train MAE 3.600 | val MAE 2.719
epoch  75 | train MAE 3.595 | val MAE 2.720
epoch  80 | train MAE 3.568 | val MAE 2.731
epoch  85 | train MAE 3.575 | val MAE 2.723
epoch  90 | train MAE 3.578 | val MAE 2.724
epoch  95 | train MAE 3.540 | val MAE 2.714


In [19]:
test_mae = run_batch(test_data, train=False)
print(f"Test MAE: {test_mae:.3f} years")

Test MAE: 3.254 years


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCNRegressor().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.L1Loss()

for epoch in range(1, 100):
    train_loss = run_batch(train_data, train=True)
    val_loss = run_batch(val_data, train=False)
    if epoch % 5 == 0:
        print(f"epoch {epoch:3d} | train MAE {train_loss:.3f} | val MAE {val_loss:.3f}")

epoch   5 | train MAE 3.660 | val MAE 2.900
epoch  10 | train MAE 3.701 | val MAE 3.003
epoch  15 | train MAE 3.642 | val MAE 2.847
epoch  20 | train MAE 3.686 | val MAE 2.936
epoch  25 | train MAE 3.623 | val MAE 2.936
epoch  30 | train MAE 3.674 | val MAE 2.948
epoch  35 | train MAE 3.618 | val MAE 2.864
epoch  40 | train MAE 3.620 | val MAE 2.939
epoch  45 | train MAE 3.662 | val MAE 2.946
epoch  50 | train MAE 3.642 | val MAE 2.872
epoch  55 | train MAE 3.607 | val MAE 2.825
epoch  60 | train MAE 3.673 | val MAE 3.051
epoch  65 | train MAE 3.639 | val MAE 2.946
epoch  70 | train MAE 3.634 | val MAE 2.924
epoch  75 | train MAE 3.614 | val MAE 3.051
epoch  80 | train MAE 3.625 | val MAE 2.891
epoch  85 | train MAE 3.643 | val MAE 3.167
epoch  90 | train MAE 3.639 | val MAE 3.188
epoch  95 | train MAE 3.641 | val MAE 3.024


In [21]:
test_mae = run_batch(test_data, train=False)
print(f"Test MAE: {test_mae:.3f} years")

Test MAE: 3.384 years
